In [1]:
from functions import *

In [2]:
print(tf.test.gpu_device_name())
print(tf.__version__)
#config = tf.compat.v1.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.compat.v1.Session(config=config)

/device:GPU:0
2.3.0


## Load and re-organize data 

In [3]:
dataset_path = 'data/segments_normWithinSubjRun_rAI.pkl'
with open(dataset_path, 'rb') as f:
    dataset = pkl.load(f)

print("Using dataset: {0}".format(dataset_path))
num_subjs = len(dataset.keys())
print("There are {num_subjs} participants in the dataset".format(num_subjs=num_subjs))

p1 = list(dataset.keys())[0]
p1_data = dataset[p1]['data']
print("Each participant's data is a {0}D array of shape {1}".format(len(p1_data.shape),
                                                                   p1_data.shape))

dataset_df = organize_dataset(dataset)

Using dataset: data/segments_normWithinSubjRun_rAI.pkl
There are 61 participants in the dataset
Each participant's data is a 3D array of shape (14, 745, 39)


## Train-test Split

In [4]:
rs = ShuffleSplit(n_splits=50, train_size=.7, random_state=0)
all_participants = dataset_df.participant.unique()

batch_size = 32

for nonTest_idx, test_idx in rs.split(all_participants):
    pass

## Cross validation

In [5]:
params = {'l2': 0, 'dropout': 0.4, 'kernel_size': 2, 'lr': 0.01, 'strides': 1}
results, results_temp, _ = cross_val_scores(dataset_df,nonTest_idx, params, k_perm=0)


CV Fold:  1
Training.....
completed!
27/27 [==============================] - 0s 16ms/step - loss: 0.6710 - acc: 0.6301
Timestep 1
Training.....
completed!
27/27 [==============================] - 0s 8ms/step - loss: 0.7217 - acc: 0.5859
Timestep 2
Training.....
completed!
27/27 [==============================] - 0s 9ms/step - loss: 0.7321 - acc: 0.5537
Timestep 3
Training.....
completed!
27/27 [==============================] - 0s 11ms/step - loss: 0.7562 - acc: 0.5573
Timestep 4
Training.....
completed!
27/27 [==============================] - 0s 12ms/step - loss: 0.7088 - acc: 0.5668
Timestep 5
Training.....
completed!
27/27 [==============================] - 0s 13ms/step - loss: 0.6991 - acc: 0.5692
Timestep 6
Training.....
completed!
27/27 [==============================] - 0s 14ms/step - loss: 0.6648 - acc: 0.6062
Timestep 7
Training.....
completed!
27/27 [==============================] - 0s 17ms/step - loss: 0.6578 - acc: 0.6193

CV Fold:  2
Training.....
completed!
27/27 [===

## Test accuracy

In [6]:
train_X, train_Y = query_dataset(dataset_df, nonTest_idx)
test_X, test_Y = query_dataset(dataset_df, test_idx)
model = classifier(train_X,train_Y,
                   params['l2'],
                   params['dropout'],
                   params['kernel_size'],
                   params['strides'],
                   params['lr'])
train_acc = model.evaluate(train_X,train_Y)[1]
test_acc = model.evaluate(test_X,test_Y)[1]
print('Train Accuracy: ',train_acc)
print('Test Accuracy: ',test_acc)

54/54 [==============================] - 1s 16ms/step - loss: 0.6521 - acc: 0.6308
Train Accuracy:  0.6981227993965149
Test Accuracy:  0.6308139562606812


## Permutation

In [11]:
results_rand = permute(train_X, train_Y, test_X, test_Y, params, k_perm=100)

--------perm = 0--------
54/54 [==============================] - 1s 14ms/step - loss: 0.7250 - acc: 0.4797
permuted train acc: 0.612
permuted val acc: 0.480
54/54 [==============================] - 1s 13ms/step - loss: 0.7371 - acc: 0.4919


## Save accuracy results and the trained model

In [13]:
with open('results/accuracy_cv_perm_rAI.pkl','wb') as f:
    pkl.dump([results, 
              results_temp,
              results_rand,
              {'train':train_acc,'test':test_acc}],f)

In [26]:
model.save("models/CustomGRU_rAI.h5")
# model = tf.keras.models.load_model("models/CustomGRU_rAI.h5") to load the model.